In [ ]:
import torch
import pandas as pd
import astropy as ap
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import pdb
from scipy.ndimage.filters import maximum_filter1d
import glob
import fitsio as fits
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data_utils
from torch.distributions import Bernoulli, Binomial, Normal, Uniform
from torch.nn.utils import clip_grad_norm_
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from torch.distributions.normal import Normal
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import math
from utils import pooling, batchify
from config import TRAIN_DATA, CV_DATA

In [ ]:
n_samples = 8200
test_size = 0.5
pool = 8

In [ ]:
train_data = pd.read_csv(TRAIN_DATA, delimiter='\t', header=None)
cv_data = pd.read_csv(CV_DATA, delimiter='\t', header=None)
df = pd.concat([train_data, cv_data], axis=0, ignore_index=True)
df = df.sample(n=n_samples)
df.drop(0, axis=1, inplace=True)
data = df.values.astype(np.float32)
data = pooling(data, (1,pool))

In [ ]:
data = torch.FloatTensor(data)

In [ ]:
data.shape

In [ ]:
sl = data.size(1); sl

In [ ]:
t = torch.FloatTensor(np.arange(0, 1, 1/sl))

In [ ]:
def subsample(y):
    p = Uniform(0.3, 0.5)
    measurements = Binomial(probs=p.sample()).sample(y.size())
    y_measure = y * measurements
    y_out = torch.stack((t, y_measure, y))
    return y_out

In [ ]:
test=subsample(data[0])

In [ ]:
test.shape

In [ ]:
dataset = []
for y in tqdm(data):
    sample = subsample(y)
    dataset.append(sample)

In [ ]:
dataset = torch.stack(dataset)

In [ ]:
dataset.shape

### Dataloaders

In [ ]:
from torch.utils.data import DataLoader
from utils import collate_ts, batchify
from loading import ToyDataset

In [ ]:
L = len(dataset)
l = int(0.5*L)

In [ ]:
train_set = dataset[:l]
cv_set = dataset[l:]

In [ ]:
train_set.size(), cv_set.size()

In [ ]:
train_loader = DataLoader(train_set, batch_size=200, collate_fn=collate_ts)
cv_loader = DataLoader(cv_set, batch_size=200, collate_fn=collate_ts)

In [ ]:
torch.save(train_loader, 'ucr_train.pt')
torch.save(cv_loader, 'ucr_cv.pt')